In [2]:
#First attempt at doing sentiment analysis. Going to use the labeled data set with the pos/neg/neutral labels

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import numpy as np
import re

In [6]:
df = pd.read_csv(r'C:\Users\Patrick\Documents\GitHub\bootcamp_capstone\kaggle_dataset\sentiment_analysis_financial_news\all-data.csv'
                , encoding = "ISO-8859-1", header=None, names=['sentiment', 'text'])

df = pd.read_csv(r'C:\Users\Patrick\Documents\GitHub\bootcamp_capstone\kaggle_dataset\stock-market_sentiment\stock_data.csv',
                  encoding = "ISO-8859-1", header=1, names=['text', 'sentiment'])
df.head()

,text,sentiment
0,user: AAP MOVIE. 55% return for the FEA/GEED i...,1
1,user I'd be afraid to short AMZN - they are lo...,1
2,MNTA Over 12.00,1
3,OI Over 21.37,1
4,PGNX Over 3.04,1


In [7]:
#remove special characters (hopefully fixes that _words, but kills websites)
#Keep alphanumeric stuff, plus spaces/punctuation, and % signs
pattern = r'[^a-zA-Z0-9\s\%]'
cleaned_buffer = []
for x in df['text']:
    cleaned_buffer.append(re.sub(pattern, '', x))
    
df['cleaned'] = cleaned_buffer

In [8]:
df.head()
#so new plan.
# First, remove special characters
# second, do lemmitization
# third, the vectorizers
# after which we can apply models

,text,sentiment,cleaned
0,user: AAP MOVIE. 55% return for the FEA/GEED i...,1,user AAP MOVIE 55% return for the FEAGEED indi...
1,user I'd be afraid to short AMZN - they are lo...,1,user Id be afraid to short AMZN they are look...
2,MNTA Over 12.00,1,MNTA Over 1200
3,OI Over 21.37,1,OI Over 2137
4,PGNX Over 3.04,1,PGNX Over 304


In [9]:
import nltk
from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

lemma_buffer = []
for x in df['cleaned']:
    lemma_buffer.append(lemmatizer.lemmatize(x))
    
df['lemma'] = lemma_buffer

In [10]:
df.head(5) #sanity check. No, I'm crazy. Everything looks the same

,text,sentiment,cleaned,lemma
0,user: AAP MOVIE. 55% return for the FEA/GEED i...,1,user AAP MOVIE 55% return for the FEAGEED indi...,user AAP MOVIE 55% return for the FEAGEED indi...
1,user I'd be afraid to short AMZN - they are lo...,1,user Id be afraid to short AMZN they are look...,user Id be afraid to short AMZN they are look...
2,MNTA Over 12.00,1,MNTA Over 1200,MNTA Over 1200
3,OI Over 21.37,1,OI Over 2137,OI Over 2137
4,PGNX Over 3.04,1,PGNX Over 304,PGNX Over 304


In [7]:
#can skip this part. Lemmings is enough
#from nltk.stem import PorterStemmer
#from nltk.tokenize import word_tokenize

#tem_buffer = []
#ps = PorterStemmer()

#for x in df['lemma']:
#    stem_buffer.append(ps.stem(x))

#df['stemmed'] = stem_buffer

In [8]:
df.head(5)

,sentiment,text,cleaned,lemma
0,neutral,"According to Gran , the company has no plans t...",According to Gran the company has no plans to...,According to Gran the company has no plans to...
1,neutral,Technopolis plans to develop in stages an area...,Technopolis plans to develop in stages an area...,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...,The international electronic industry company ...,The international electronic industry company ...
3,positive,With the new production plant the company woul...,With the new production plant the company woul...,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...,According to the company s updated strategy fo...,According to the company s updated strategy fo...


In [9]:
#remove stop words
#do more exploration stuff first. Word Clouds?

In [10]:
labels = df['sentiment']
print(labels)
xtrain, xtest, ytrain, ytest = train_test_split(df['lemma'], labels, test_size=.4, random_state=10)
#something's wrong here?

0        neutral
1        neutral
2       negative
3       positive
4       positive
          ...   
4841    negative
4842     neutral
4843    negative
4844    negative
4845    negative
Name: sentiment, Length: 4846, dtype: object


In [28]:
xtrain.shape


(2907, 62922)

In [29]:
ytrain.shape

(2907,)

In [12]:
cv = CountVectorizer(stop_words='english')
count_train = cv.fit_transform(xtrain.values)
count_test = cv.transform(xtest.values)
print(cv.get_feature_names_out())
#lots of numbers here.

['00' '000' '000063' ... 'zoo' 'zte' 'zu']


In [13]:
#convert cv to dataframe
count_df = pd.DataFrame(data=count_train.A, columns=cv.get_feature_names_out())
count_df.head()

#this will be a sparse matrix (so all the zeroes are fine)
#columns are the words
#the rows are the number of times that word appears in the string (row X corresponds to string X)

,00,000,000063,001,003,0030,004,005,006,01,...,zepro,zero,zinc,zinclead,zloty,zoltan,zone,zoo,zte,zu
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#tfidf version
#this will stop the common words from dominating (suppresses them and gives lowers numbers)
from sklearn.feature_extraction.text import TfidfVectorizer
tfid_v = TfidfVectorizer(stop_words='english', max_df = .7, ngram_range=(1,3), max_feature=10000) 
#play around with ngram range to capture different numbers of words as a column in the tfidf vector
#also play with max features (the shape of xtrain tells you how many features)
tfid_train = tfid_v.fit_transform(xtrain.values) #scikit learn 
tfid_test = tfid_v.transform(xtest.values)
print(tfid_v.get_feature_names_out())

['00' '00 aldata' '00 aldata 100' ... 'zte corp 000063' 'zu'
 'zu beeinflussen']


In [15]:
xtrain = tfid_train #replace the current xtrain set with my vecotrized version.

In [16]:
#create dataframes of tfidf vector
tfid_df = pd.DataFrame(data=tfid_train.A, columns=tfid_v.get_feature_names_out())
tfid_df.head()
#weird. it's doubles, while the above is ints
#it looks like this fails because I need to have the headliens tokenized into words first before applying stemming/lemmings


#so this needs to be fed into the nb_classifer as my X. The TFIDF converts strings of text into numerical values, which
#can then be applied to the model (takes in numerics, to output numerics. or something like that)


,00,00 aldata,00 aldata 100,00 mn,00 mn eur,000,000 2009,000 2009 lifted,000 company,000 company added,...,zone neudorf st,zone planned,zone planned invest,zoo,zoo poland,zte,zte corp,zte corp 000063,zu,zu beeinflussen
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
#xtrain, xtest, ytrain, ytest = train_test_split(df['lemma'], labels, test_size=.4, random_state=10)
xtrain1, xtest1, ytrain1, ytest1 = train_test_split(tfid_df, tfid_df, test_size=.4, random_state=10)

In [27]:
xtrain

<2907x62922 sparse matrix of type '<class 'numpy.float64'>'
	with 98271 stored elements in Compressed Sparse Row format>

In [24]:
#now tdo the bayes classifer (already imported)
nb_class = MultinomialNB()

#nb_class.fit(len(xtrain), ytrain) #the original line
#nb_class.fit(len(xtrain), np.reshape(ytrain, -1) ) #this wants an array, but it created as series?
#nb_class.fit(len(xtrain), ytrain_1)
nb_class.fit(xtrain, ytrain)
nb_class.fit(tfidf_data, labels_off_original_data) #do this.

#think about using fit_transform, because i'm certainly not right now.
#for dataset m by n:
    #tfidf this will be m by t, where t is the nubmer of words in the text for row m
    #pass to 

predict = nb_class.predict(xtest1)
score = metrics.accuracy_score(ytest, predict)

NameError: name 'tfidf_data' is not defined